# Séries de tempo  
    
### 1. O que torna a série temporal especial?
Como o nome sugere, TS é uma coleção de pontos de dados coletados em intervalos de tempo constantes . Estes são analisados para determinar a tendência de longo prazo, de modo a prever o futuro ou realizar alguma outra forma de análise. Mas o que faz um TS diferente de dizer um problema de regressão regular? Existem duas coisas:

* É dependente do tempo . Portanto, a suposição básica de um modelo de regressão linear que as observações são independentes não vale neste caso. 
* Juntamente com uma tendência crescente ou decrescente, a maioria dos TS tem alguma forma de tendências de sazonalidade , ou seja, variações específicas de um determinado período de tempo. Por exemplo, se você ver as vendas de uma jaqueta de lã ao longo do tempo, você invariavelmente encontrará maiores vendas nas temporadas de inverno.


Por causa das propriedades inerentes de um TS, existem várias etapas envolvidas na análise. Estes são discutidos em detalhes abaixo. Vamos começar carregando um objeto TS no Python. Nós estaremos usando o popular conjunto de dados AirPassengers.

### 2. Carregando e Manipulando Séries Temporais em Pandas

O Pandas possui bibliotecas dedicadas para manipular objetos TS, particularmente a classe datatime64 [ns]  que armazena informações de tempo e nos permite executar algumas operações realmente rápidas. Vamos começar ativando as bibliotecas necessárias.

### 3. Carregando os dados

Para acesso ao link que dá acesso ao DataSet [clique aqui](https://www.kaggle.com/rakannimer/air-passengers)

Os dados contêm um mês e um número específicos de passageiros viajando naquele mês. Mas isso ainda não é lido como um objeto TS, pois os tipos de dados são 'objeto' e 'int'. Para ler os dados como uma série temporal, temos que passar argumentos especiais para o comando read_csv:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import sqlite3
%matplotlib inline


plt.rcParams ['figure.figsize'] = 15, 6


dbfile = "../../99 Datasets/datasets.db"
tabela = "kaggle_air_passengers"

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m')



db = sqlite3.connect(dbfile)
data = pd.read_sql_query(f'select * from "{tabela}"', con=db)
# data = pd.read_csv('./data/AirPassengers.csv', parse_dates=['Month'], index_col='Month',date_parser=dateparse)

data.Month=pd.to_datetime(data.Month, format='%Y-%m')
data.set_index('Month', inplace=True)
data.head()

In [ ]:
data.shape

Vamos entender os argumentos um por um:

- 1) **parse_dates** : especifica a coluna que contém as informações de data e hora. Como dissemos acima, o nome da coluna é 'Mês'.

- 2) **index_col**: Uma idéia-chave por trás do uso de Pandas para dados TS é que o índice deve ser a variável que descreve as informações de data e hora. Então este argumento diz aos pandas para usarem a coluna 'Mês' como índice.

- 3) **date_parser**: especifica uma função que converte uma string de entrada em uma variável datetime. Por padrão, o Pandas lê dados no formato 'AAAA-MM-DD HH: MM: SS'. Se os dados não estiverem nesse formato, o formato deverá ser definido manualmente. Algo parecido com a função de busca de dados definida aqui pode ser usado para este propósito.

Agora podemos ver que os dados têm objeto de tempo como índice e #Passengers como a coluna. Podemos cruzar o tipo de dados do índice com o seguinte comando:

In [ ]:
data.index

Observe o dtype = 'datetime [ns]', que confirma que é um objeto datetime. Como uma preferência pessoal, eu converteria a coluna em um objeto de série para evitar referir nomes de colunas toda vez que eu usar o TS. Por favor, sinta-se livre para usar como um dataframe é que funciona melhor para você.

### 3. Como verificar a estacionariedade de uma série temporal?

Uma TS é considerada estacionária se suas propriedades estatísticas , como média, variância permanecer constante ao longo do tempo . Mas por que isso é importante? A maioria dos modelos TS trabalha no pressuposto de que o TS é estacionário. Intuitivamente, podemos afirmar que se um TS tem um comportamento particular ao longo do tempo, existe uma probabilidade muito alta de que ele siga o mesmo no futuro. Além disso, as teorias relacionadas a séries estacionárias são mais maduras e mais fáceis de implementar em comparação com séries não estacionárias.

A estacionariedade é definida usando um critério muito estrito. No entanto, para fins práticos, podemos supor que a série seja estacionária se ela tiver propriedades estatísticas constantes ao longo do tempo, isto é. Os seguintes:

- média constante
- variância constante
- uma autocovariância que não depende do tempo.

Vou pular os detalhes, pois é muito claramente definido neste artigo . Vamos para as maneiras de testar a estacionariedade. Primeiro e mais importante é simples traçar os dados e analisar visualmente. Os dados podem ser plotados usando o seguinte comando:

In [ ]:
plt.plot(data)

É claramente evidente que há uma tendência geral crescente nos dados, juntamente com algumas variações sazonais. No entanto, nem sempre é possível fazer tais inferências visuais (veremos esses casos mais tarde). Então, mais formalmente, podemos verificar a estacionariedade usando o seguinte:

- **Plotando Rolando Estatísticas**: Podemos traçar a média móvel ou variância móvel e ver se varia com o tempo. Ao mover a média / variância, quero dizer que, em qualquer instante, 't', consideraremos a média / variância do último ano, isto é, os últimos 12 meses. Mas, novamente, isso é mais uma técnica visual.


- **Teste Dickey-Fuller**: Este é um dos testes estatísticos para verificar a estacionariedade. Aqui a hipótese nula é que o TS é não-estacionário. Os resultados do teste compreendem uma estatística de teste e alguns  valores críticos para os níveis de confiança de diferença. Se a estatística de teste for menor que o valor crítico, podemos rejeitar a hipótese nula e dizer que a série é estacionária. Consulte este artigo para detalhes.

Esses conceitos podem não parecer muito intuitivos neste momento. Eu recomendo passar pelo artigo prequel. Se você estiver interessado em alguma estatística teórica, você pode consultar Introdução a Séries Temporais e Previsão  por Brockwell e Davis . O livro é um pouco pesado para as estatísticas, mas se você tiver a habilidade de ler as entrelinhas, poderá entender os conceitos e tangenciar as estatísticas tangencialmente.

De volta à verificação da estacionariedade, usaremos muito os gráficos estatísticos de rolagem juntamente com os resultados do teste Dickey-Fuller, de modo que defini uma função que recebe um TS como entrada e os gerou para nós. Por favor, note que eu tracei desvio padrão ao invés de variância para manter a unidade semelhante à média.

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
#     print('Results of Dickey-Fuller Test:')
#     dftest = adfuller(timeseries, autolag='AIC')
#     dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
#     for key,value in dftest[4].items():
#         dfoutput['Critical Value (%s)'%key] = value
#     print(dfoutput)

O código é bem direto. Por favor, sinta-se à vontade para discutir o código nos comentários se você enfrentar desafios ao compreendê-lo. Vamos executá-lo para nossas séries de entrada:

In [ ]:
test_stationarity(data)

### 4. Como fazer uma série temporal estacionária?

Embora a suposição de estacionariedade seja tomada em muitos modelos TS, quase nenhuma das séries temporais práticas é estacionária. Então, os estatísticos descobriram maneiras de tornar as séries estacionárias, o que discutiremos agora. Na verdade, é quase impossível fazer uma série perfeitamente estacionária, mas tentamos levá-la o mais perto possível. Vamos entender o que está tornando um TS não estacionário. Existem duas razões principais por trás do não estacionarista de um TS: 

- 1. Tendência - variação média ao longo do tempo. Por exemplo, neste caso, vimos que, em média, o número de passageiros estava crescendo com o tempo. 
- 2. Sazonalidade - variações em intervalos de tempo específicos. Por exemplo, as pessoas podem ter a tendência de comprar carros em um determinado mês devido ao aumento de salário ou festivais.


O princípio subjacente é modelar ou estimar a tendência e a sazonalidade da série e removê-la da série para obter uma série estacionária. Em seguida, as técnicas de previsão estatística podem ser implementadas nesta série. A etapa final seria converter os valores previstos na escala original aplicando as restrições de tendência e sazonalidade. Nota: discutirei vários métodos. Alguns podem funcionar bem neste caso e outros não. Mas a ideia é pegar um jeito de todos os métodos e não focar apenas no problema em questão. Vamos começar trabalhando na parte da tendência. 

### Estimando e eliminando tendências 

Um dos primeiros truques para reduzir a tendência pode ser a transformação. Por exemplo, neste caso, podemos ver claramente que há uma tendência positiva significativa. Assim, podemos aplicar transformações que penalizam valores mais altos do que valores menores. Estes podem ser um log, raiz quadrada, raiz cúbica, etc. Vamos pegar uma transformação log aqui para simplificar:


In [ ]:
ts_log = np.log(data)
plt.plot(ts_log)

Neste caso mais simples, é fácil ver uma tendência de avanço nos dados. Mas não é muito intuitivo na presença de ruído. Assim, podemos usar algumas técnicas para estimar ou modelar essa tendência e, em seguida, removê-la da série. Pode haver muitas maneiras de fazer isso e algumas das mais usadas são: 

- Agregação - tomando a média por um período de tempo como médias mensais / semanais
- Suavização - tendo médias móveis 
- Ajuste polinomial - ajuste um modelo de regressão 

Suavização refere-se a estimativas contínuas, ou seja, considerando os últimos instantes. Existem várias maneiras, mas vou discutir duas delas aqui. 

### Média móvel

Nesta abordagem, tomamos a média de valores consecutivos de "k" dependendo da frequência das séries temporais. Aqui podemos tirar a média dos últimos 1 ano, ou seja, últimos 12 valores. Os pandas têm funções específicas definidas para determinar as estatísticas de rolagem.

In [ ]:
moving_avg = ts_log.rolling(12).mean()
plt.plot(ts_log)
plt.plot(moving_avg, color='red')

A linha vermelha mostra a média de rolamento. Vamos subtrair isso da série original. Observe que, como estamos tirando a média dos últimos 12 valores, a média de rolagem não está definida para os primeiros 11 valores. Isso pode ser observado como:

In [ ]:
ts_log_moving_avg_diff = ts_log - moving_avg
ts_log_moving_avg_diff.head(16)

Observe os 11 primeiros sendo Nan. Vamos descartar esses valores NaN e verificar os gráficos para testar a estacionariedade.

In [ ]:
ts_log_moving_avg_diff.dropna(inplace=True)
test_stationarity(ts_log_moving_avg_diff)

Isto parece uma série muito melhor. Os valores de rolagem parecem estar variando um pouco, mas não há uma tendência específica.

No entanto, uma desvantagem nessa abordagem particular é que o período de tempo deve ser estritamente definido. Neste caso, podemos obter médias anuais, mas em situações complexas, como a previsão de um preço de ações, é difícil encontrar um número. Então, pegamos uma "média móvel ponderada", em que valores mais recentes recebem um peso maior. Pode haver muitas técnicas para atribuir pesos.

O próximo tópico seria como **Eliminar tendência e sazonalidade** mas vamos deixa-lo de lado para evitar a parte complicada do tema e quem está confortavel e precisa evoluir no assunto, os dois subtópicos seriam: Diferenciação & Decomposição

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts_log)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(ts_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

### Forecasting

Familia ARIMA

Deixe-me dar uma breve introdução ao ARIMA . Não vou entrar nos detalhes técnicos, mas você deve entender esses conceitos detalhadamente se quiser aplicá-los com mais eficiência. ARIMA significa Médias Móveis Integradas Auto-Regressivas . A previsão ARIMA para uma série temporal estacionária nada mais é que uma equação linear (como uma regressão linear). Os preditores dependem dos parâmetros (p, d, q) do modelo ARIMA:


Algumas palavras sobre o modelo. Letra por letra, construiremos o nome completo - ARIMA (p, d, q).


- **AR (p)** - modelo de autorregressão, isto é, regressão da série temporal em si. Premissa básica - os valores da série atual dependem de seus valores anteriores com algum atraso (ou várias defasagens). O atraso máximo no modelo é referido como p. Para determinar o p inicial. (em econometria analisamos um grafico chamado PACF para anlisar o p, em machine learning faremos via grid search).

- ** MA (q)** - modelo de média móvel. Sem entrar em detalhes, modela o erro da série temporal, novamente a suposição é - o erro atual depende do anterior com algum atraso, que é referido como q. (em econometria usamos o grafico ACF).

Vamos fazer uma pequena pausa e combinar as primeiras 4 letras:

**AR(p) + MA(q) = ARMA(p,q)**

O que temos aqui é o modelo de médio movimento autorregressivo! Se a série é estacionária, pode ser aproximada com essas 4 letras. Devemos continuar?

- **I(d)**— ordem de integração. É simplesmente o número de diferenças não sazonais necessárias para tornar a série estacionária. Como a ideia de estacionariedade é razoavelmente complicada para essa introdução, vamos defini-la apenas como uma curva "bem comportada", que em séries temporais seria algo não explosivo, com variância constante e sazonalidade constante.

**AR(p) + I(d) + MA(q) = ARIMA(p,d,q)**

Há outros filtros como o S de sazonal e outras formas funcionais como o VAC e o VEC e até modelos que preevem volatidade (muito usado no mercado financeiro) como os GARCH. Aqui ficaremos no mais simples.


Uma preocupação importante aqui é como determinar o valor de 'p' e 'q'. Nós usamos dois gráficos para determinar esses números. Vamos discuti-los primeiro.

**Função de Autocorrelação (ACF)**: É uma medida da correlação entre o TS com uma versão defasada de si mesmo. Por exemplo, no intervalo 5, o ACF compararia as séries no instante de tempo 't1' ... 't2' com as séries no instante 't1-5'… 't2-5' (t1-5 e t2 sendo pontos finais).


**Função de Autocorrelação Parcial (PACF)**: Mede a correlação entre a TS com uma versão defasada de si mesma, mas depois elimina as variações já explicadas pelas comparações intervenientes. Por exemplo, no lag 5, ele verificará a correlação, mas removerá os efeitos já explicados pelos lags 1 a 4.

In [ ]:
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
ts_log_diff = ts_log - ts_log.shift()
ts_log_diff = ts_log_diff.dropna()
lag_acf = acf(ts_log_diff, nlags=20)
lag_pacf = pacf(ts_log_diff, nlags=20, method='ols')

In [ ]:
#Plot ACF: 
plt.rcParams ['figure.figsize'] = 20, 6

plt.subplot(121) 
plt.plot(lag_acf)
plt.xticks(np.arange(20))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

In [ ]:
#Plot PACF:
plt.rcParams ['figure.figsize'] = 20, 6

plt.subplot(122)
plt.plot(lag_pacf)
plt.xticks(np.arange(20))
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_log_diff)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

Neste gráfico, as duas linhas pontilhadas em ambos os lados de 0 são os intervalos de confiança. Estes podem ser usados para determinar os valores 'p' e 'q' como:

p - O valor de retardo em que o gráfico PACF cruza o intervalo de confiança superior pela primeira vez. Se você notar de perto, neste caso p = 2.

q - O valor de retardo no qual o gráfico ACF cruza o intervalo de confiança superior pela primeira vez. Se você notar de perto, neste caso, q = 2.

Agora, vamos criar 3 modelos ARIMA diferentes, considerando efeitos individuais e combinados. Também vou imprimir o RSS para cada um. Por favor, note que aqui RSS é para os valores dos resíduos e não séries reais.

Precisamos carregar o modelo ARIMA primeiro:

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
# MODELO AR
model = ARIMA(ts_log, order=(2, 1, 0))  
results_AR = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='red')

In [ ]:
# MODELO MA

model = ARIMA(ts_log, order=(0, 1, 5))  
results_MA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_MA.fittedvalues, color='red')

In [ ]:
# Combinando AR + MA 

model = ARIMA(ts_log, order=(3, 1, 3))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')

In [ ]:
### Sazonalidade
import statsmodels.api as sm
mod = sm.tsa.statespace.SARIMAX(ts_log,
                                order=(2, 1, 2),
                                seasonal_order=(1, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])


In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
results.plot_diagnostics(figsize=(16, 8))
plt.show()

## Fazendo testes


Exemplo do crossvalidation com timeseries
<img src='./imgs/crossvalidation.png'>

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('1955-01-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = ts_log['1949':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Passengers')
plt.legend()
plt.show()

In [ ]:
pred_uc = results.get_forecast(steps=100)
pred_ci = pred_uc.conf_int()
ax = ts_log.plot(label='observed', figsize=(14, 7))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Passengers')
plt.legend()
plt.show()